In [ ]:
"""
数据预处理
使用数据工程方法，将比特币价格数据归一化为近似周期性的数据。
"""

In [100]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import timedelta

# -------------------------------
# 1. 加载数据
# 假设CSV文件 'btc_prices.csv' 中包含 Date 和 Close 两列
df = pd.read_csv('btc_prices.csv', parse_dates=['open_date'], index_col='open_date')
df = df.sort_index()  # 确保按日期排序

# 将日期转换为数值（天数）用于回归分析
df['Time'] = np.arange(len(df))

In [101]:
# 比特币价格

# 创建图表
fig = make_subplots()

# 添加实际价格曲线
fig.add_trace(go.Scatter(x=df.index, y=df['open_price'], mode='lines', name='比特币价格'))

# 更新布局
fig.update_layout(
    title='比特币价格',
    xaxis_title='日期',
    yaxis_title='价格',
    legend=dict(x=0, y=1)
)

# 显示图表
fig.show()

In [102]:
# -------------------------------
# 2. 对价格取对数（缓解异方差性）
# df['Log_Close'] = np.log(df['open_price'])
df['Log_Close'] = df['open_price']

# -------------------------------
# 3. 去趋势处理
# 使用30天移动平均来估计趋势，得到“去趋势”后的数据
window = 30

def custom_moving_average(series, window):
    half_window = window // 2
    moving_avg = []

    for i in range(len(series)):
        start_idx = max(0, i - half_window)
        end_idx = min(len(series), i + half_window + 1)
        window_data = series[start_idx:end_idx]
        window_avg = window_data.mean()
        moving_avg.append(window_avg)

    return pd.Series(moving_avg, index=series.index)

# df['Moving_Avg'] = df['Log_Close'].rolling(window=window, center=True).mean()
df['Moving_Avg'] = custom_moving_average(df['Log_Close'], window)

# 创建图表
fig = make_subplots()

# 添加实际价格曲线
fig.add_trace(go.Scatter(x=df.index, y=df['Log_Close'], mode='lines', name='Log价格'))

# 添加中心移动平均价格曲线
fig.add_trace(go.Scatter(x=df.index, y=df['Moving_Avg'], mode='lines', name='Log中心移动平均价格'))

# 更新布局
fig.update_layout(
    title='比特币Log价格中心移动平均',
    xaxis_title='日期',
    yaxis_title='Log价格',
    legend=dict(x=0, y=1)
)

# 显示图表
fig.show()

In [103]:
# 一阶线性回归
X = df[["Time"]]  # 自变量：天数
# y = df["Log_Close"]   # 因变量：价格
y = df["Moving_Avg"]   # 因变量：价格
model = LinearRegression()
model.fit(X, y)

# 预测值
df["Predicted_Price"] = model.predict(X)

# 创建图表
fig = make_subplots()

# 添加实际价格曲线
fig.add_trace(go.Scatter(x=df.index, y=df['Log_Close'], mode='lines', name='Log价格'))

# 添加中心移动平均价格曲线
fig.add_trace(go.Scatter(x=df.index, y=df['Moving_Avg'], mode='lines', name='Log中心移动平均价格'))

# 添加预测价格曲线
fig.add_trace(go.Scatter(x=df.index, y=df['Predicted_Price'], mode='lines', name='回归Log价格'))

# 更新布局
fig.update_layout(
    title='比特币Log价格的一阶线性回归',
    xaxis_title='日期',
    yaxis_title='Log价格',
    legend=dict(x=0, y=1)
)

# 显示图表
fig.show()

In [104]:
# -------------------------------
# 3. 去趋势处理
# 使用30天移动平均来估计趋势，得到“去趋势”后的数据
df['Detrended'] = df['Log_Close'] - df['Predicted_Price']

# -------------------------------
# 4. 归一化处理
# 方法1：Z-score 标准化
scaler_z = StandardScaler()
df['Zscore'] = scaler_z.fit_transform(df[['Detrended']])

# 方法2：Min-Max 归一化到 [0,1]
scaler_mm = MinMaxScaler(feature_range=(0, 1))
df['MinMax'] = scaler_mm.fit_transform(df[['Detrended']])

# -------------------------------
# 5. 使用 Plotly 绘制图形
# 创建两个子图：上图显示 Log_Close 与趋势（Moving_Avg），下图显示去趋势数据及归一化结果
fig = make_subplots()

# 去趋势数据及归一化结果
fig.add_trace(go.Scatter(x=df.index, y=df['Detrended'], mode='lines',
                         name='Detrended Log Price', line=dict(color='purple')),)
fig.add_trace(go.Scatter(x=df.index, y=df['Zscore'], mode='lines',
                         name='Z-score Normalized', line=dict(color='green')))
fig.add_trace(go.Scatter(x=df.index, y=df['MinMax'], mode='lines',
                         name='Min-Max Normalized', line=dict(color='red')))
# 更新布局
fig.update_layout(
    title='去趋势以及归一化结果',
    xaxis_title='日期',
    yaxis_title='Log价格',
    legend=dict(x=0, y=1)
)

fig.show()

In [109]:
# 傅里叶变换+输出频谱图
# 2. 使用 FFT 进行傅里叶变换和低通滤波

# 计算 FFT 变换
fft_coeffs = np.fft.fft(df['Detrended'])

# 3. 计算频率轴
sampling_rate = 1  # 采样频率
n = len(df)
frequencies = np.fft.fftfreq(n, d=1/sampling_rate)

# 计算对应频率
freqs = np.fft.fftfreq(len(df))

# 4. 计算幅度谱
amplitude = np.abs(fft_coeffs)

# 5. 绘制频谱图
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=frequencies[:n//2],  # 仅显示正频率部分
    y=amplitude[:n//2],
    mode='lines',
    name='Amplitude Spectrum'
))
fig.update_layout(
    title='比特币价格频谱图',
    xaxis_title='频率 (Hz)',
    yaxis_title='幅度',
    template='plotly_dark'
)
fig.show()

In [110]:
# 低通滤波
# 取正频率部分
positive_freq = frequencies > 0
xf = frequencies[positive_freq]
yf = fft_coeffs[positive_freq]

# 自适应阈值选择：基于累积能量占比
power_threshold = 0.1 # 保留能量的比例
spectrum_power = np.abs(yf) ** 2  # 频谱能量
cumulative_energy = np.cumsum(spectrum_power) / np.sum(spectrum_power)  # 累积能量占比
threshold_index = np.argmax(cumulative_energy > power_threshold)
threshold_freq = xf[threshold_index]  # 对应的频率阈值

# threshold_freq = 0.004

print("threshold_freq:",threshold_freq)

# 设置低通滤波的截止频率（这里 cutoff 为相对频率阈值，可调）
# cutoff = 0.001
cutoff = threshold_freq

# 对高于截止频率的成分置零
filtered_fft = fft_coeffs.copy()
filtered_fft[np.abs(freqs) > cutoff] = 0

# 利用逆 FFT 得到滤波后的对数价格信号（趋势）
filtered_log = np.fft.ifft(filtered_fft).real

# 将滤波后的对数价格转换为实际价格
# filtered_price = np.exp(filtered_log)
filtered_price = filtered_log

# -------------------------------
# 3. 对滤波后（低频趋势）的信号进行预测
# 这里采用线性回归在对数价格（趋势）上进行外推预测
# 构造 DataFrame 保存滤波后的数据
trend_df = pd.DataFrame({
    'Filtered_Log': filtered_log,
    'Filtered_Price': filtered_price
}, index=df.index)

# 用时间（天数）作为自变量
trend_df['Time'] = (trend_df.index - trend_df.index[0]).days
X = trend_df['Time'].values.reshape(-1, 1)
y = trend_df['Filtered_Log'].values

# 拟合线性回归模型
lr_model = LinearRegression()
lr_model.fit(X, y)

# -------------------------------
# 4. 绘图展示结果（使用 Plotly）
fig = go.Figure()
# 原始数据
fig.add_trace(go.Scatter(x=df.index, y=df['Detrended'], mode='lines', name=' Price'))
# 低通滤波后的趋势
fig.add_trace(go.Scatter(x=trend_df.index, y=trend_df['Filtered_Log'], mode='lines', name='Filtered Trend'))

fig.update_layout(
    title='BTC Price Forecast using FFT Low-Pass Filtering and Linear Regression',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)
fig.show()

threshold_freq: 0.0007291286912139992


In [96]:
# 傅里叶变换结果+一阶线性回归逆变换+指数逆变换
fft_filtered_log = filtered_log # 傅里叶变换后的log价格
fft_filtered_trended_log = filtered_log + df["Predicted_Price"] # 傅里叶变换 + 一阶线性回归逆变换 后的log价格
# fft_filtered_trended_price = np.exp(fft_filtered_trended_log) # 傅里叶变换 + 一阶线性回归逆变换 后的实际价格
fft_filtered_trended_price = fft_filtered_trended_log # 傅里叶变换 + 一阶线性回归逆变换 后的实际价格

# 绘图展示结果（使用 Plotly）
fig = go.Figure()
# FFT+低通滤波后的价格
fig.add_trace(go.Scatter(x=df.index, y=fft_filtered_trended_price, mode='lines', name='FFT Price'))
# 添加实际价格曲线
fig.add_trace(go.Scatter(x=df.index, y=df['open_price'], mode='lines', name='比特币价格'))

fig.update_layout(
    title='BTC Price using FFT Low-Pass Filtering',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)
fig.show()

In [38]:

# -------------------------------
# 6. 逆归一化示例
# 假设经过后续预测得到 Min-Max 归一化后的未来数据（示例数据）
forecast_norm = np.array([0.45, 0.47, 0.50, 0.52, 0.55])
# 逆 Min-Max 归一化：将归一化值还原到去趋势数据尺度
forecast_detrended = scaler_mm.inverse_transform(forecast_norm.reshape(-1, 1)).flatten()

# 逆去趋势：将预测的去趋势数据加上最后一天的移动平均（简单假设未来趋势与最后值一致）
last_mavg = df['Moving_Avg'].iloc[-1]
forecast_log = forecast_detrended + last_mavg
# 逆对数转换：取指数还原为价格
# forecast_price = np.exp(forecast_log)
forecast_price = forecast_log
print("预测的未来价格:", forecast_price)

# 可将预测结果与原始数据绘制在一张图中：
last_date = df.index[-1]
future_dates = [last_date + timedelta(days=i) for i in range(1, len(forecast_price) + 1)]
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df.index, y=df['open_price'], mode='lines', name='Original Price'))
fig2.add_trace(go.Scatter(x=future_dates, y=forecast_price, mode='lines', name='Forecast Price'))
fig2.update_layout(title="BTC Price Forecast (Inverse Normalization)",
                   xaxis_title="Date", yaxis_title="Price (USD)", template="plotly_white")
fig2.show()



预测的未来价格: [ 97275.64149169 101574.18142766 108380.75511515 113170.02195532
 120753.64293879]
